<a href="https://colab.research.google.com/github/heysourin/Generative-AI-101/blob/main/RAG%20Apps/Llama_Index_Mistral_HuggingfaceEmbedding/RAG_Application_Using_LlamaIndex_and_MistralAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install llama-index-llms-huggingface llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 36.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 M

In [4]:
!mkdir data

In [5]:
from google.colab import files
uploaded = files.upload()

Saving chameleon.pdf to chameleon.pdf


In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [8]:
# Load documents
documents = SimpleDirectoryReader("./data/").load_data()
documents

[Document(id_='8132c574-1c43-43ff-977b-69e5fdb31554', embedding=None, metadata={'page_label': '1', 'file_name': 'chameleon.pdf', 'file_path': '/content/data/chameleon.pdf', 'file_type': 'application/pdf', 'file_size': 25242519, 'creation_date': '2024-06-05', 'last_modified_date': '2024-06-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Chameleon: Mixed-Modal Early-Fusion Foundation\nModels\nChameleon Team1,∗\n1FAIR at Meta\n∗See Contributions section for full author list.\nWe present Chameleon, a family of early-fusion token-based mixed-modal models capable of under-\nstanding and generating images and text in any arbitrary sequence. We outline a stable training\napproach from inception, an alignment recipe, and an architectural parameterization ta

## Huggingface logging and accessing Mistral:

In [9]:
from google.colab import userdata
HUGGING_FACE_API = userdata.get('HUGGING_FACE_API')

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

In [11]:
# Log in with your Hugging Face token
login(token=HUGGING_FACE_API)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Attempt to load the model:
# You have to go to: https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1
# And confirm your access

try:
  tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
  model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
  print("Model loaded successfully.")
except Exception as e:
  print(f"Failed to load the model: {e}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
# Setup prompts
from llama_index.core import PromptTemplate

system_prompt = """
<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [13]:
import torch

llm = HuggingFaceLLM(
  context_window=4096, # How many max tokens you will pass to the model
  max_new_tokens=256, # Max number of tokens you want to generate
  generate_kwargs={"temperature": 0.7, "do_sample": False},
  system_prompt=system_prompt,
  query_wrapper_prompt=query_wrapper_prompt, #
  tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
  model_name="mistralai/Mistral-7B-Instruct-v0.1",
  device_map="auto",
  stopping_ids=[50278, 50279, 50277, 1, 0],
  tokenizer_kwargs={"max_length": 4096},
  # uncomment this if using CUDA to reduce memory usage
  model_kwargs={"torch_dtype": torch.float16}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [14]:
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor
# !pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.7.0
    Uninstalling sentence-transformers-2.7.0:
      Successfully uninstalled sentence-transformers-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.2.1 requires sentence-transformers<3.0.0,>=2.6.1, but you have sentence-transformers 3.0.0 which is incompatible.
llama-index-embeddings-instructor 0.1.3 requires sentence-transformers<3.0.0,>=2.2.2, but you have sentence-transformers 3.0.0 which is incompatible.


In [15]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
from llama_index.core import VectorStoreIndex, ServiceContext

service_context = ServiceContext.from_defaults(
  chunk_size=1024,
  llm=llm,
  embed_model=embed_model,
)

<ipython-input-16-ee3dc37b69ab>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [17]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [18]:
query_engine = index.as_query_engine()

In [19]:
query0 = query_engine.query("What is a chameleon model?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [21]:
print(query0.response)

A chameleon model is a type of artificial intelligence (AI) model that is designed to generate text and images based on a given prompt. It is trained on a large dataset of text and images and uses a combination of natural language processing (NLP) and computer vision techniques to generate responses. The model is capable of generating text and images that are similar to those in the training data and can be used for a variety of tasks, such as text generation, image generation, and language translation.
